In [273]:
import pandas as pd
import numpy as np

In [274]:
data = pd.read_csv('/Users/tanhoangminhco/Documents/Coding/Python/Machine Learning/datasets/iris.csv')

In [275]:
class Node():
    def __init__(self, threshold=None, key_feature=None, ig=None, left=None, right=None, decision=None):
        #only for condition node
        self.threshold = threshold
        self.key_feature = key_feature
        self.left = left
        self.right = right
        self.ig = ig
        
        #only for leaf
        self.decision = decision

In [276]:
class DecisionTree():
    def __init__(self, min_samples_split=2 ,max_layers=2):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_layers = max_layers
        self.layers = 0
        
    def fit(self, dataset):
        data = dataset.values
        self.root = self.build_tree(data ,current_layer=0)#When layer(depth)=0, it means root Node
    
    def build_tree(self, dataset, current_layer):
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        if (num_samples >= self.min_samples_split and current_layer<=self.max_layers):
            best_split = self.best_split(dataset, num_features)
            self.layers = current_layer
            if best_split['information_gain']>0:
                left_subtree = self.build_tree(best_split["leftData"], current_layer+1)
                right_subtree = self.build_tree(best_split["rightData"], current_layer+1)
                return Node(best_split['threshold'], best_split['feature_index'], best_split['information_gain'],
                            left_subtree, right_subtree)
        
        decision = self.calculate_leaf_value(Y)
        return Node(decision=decision)
            
    def best_split(self, dataset, num_features):
        best_split = {}
        maxIG = -float('inf')
    
        for feature_index in range(num_features):
            thresholds = np.unique(dataset[:, feature_index])
            
            for threshold in thresholds:
                
                dataleft = np.array([row for row in dataset if row[feature_index]<=threshold])
                dataright = np.array([row for row in dataset if row[feature_index]>threshold])
                if (len(dataleft)>0) and (len(dataright)>0): #Without this [:,-1] is impossible
                    information_gain = self.information_gain(dataset[:,-1], dataleft[:,-1], dataright[:,-1])
                    if (information_gain > maxIG):
                        maxIG = information_gain
                        best_split['feature_index'] = feature_index
                        best_split['threshold'] = threshold
                        best_split['information_gain'] = information_gain
                        best_split['leftData'] = dataleft
                        best_split['rightData'] = dataright
        
        return best_split
    
    def entropy(self, dataset):
        entropy = 0
        labels = np.unique(dataset)
        for label in labels:
            ratio = len(dataset[dataset==label])/len(dataset)
            entropy += ratio * np.log2(ratio)
        return -entropy

    def information_gain(self, parent, left, right):
        weightL = len(left) / len(parent)
        weightR = len(right) / len(parent)
        return self.entropy(parent) - weightL*self.entropy(left) - weightR*self.entropy(right)
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def predict(self,X):
        return [self.predictHelper(self.root, x) for x in X]

    
    def predictHelper(self, cur, x):
        if (cur.decision):
            return cur.decision
        key = x[cur.key_feature]
        if key <= cur.threshold:
            return self.predictHelper(cur.left, x)
        if key > cur.threshold:
            return self.predictHelper(cur.right, x)
    
    def precision(self, tree, actual):
        correct = 0
        for i in range(len(tree)):
            if tree[i] == actual[i]:
                correct += 1
        return correct/len(tree)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.decision is not None:
            print(tree.decision)

        else:
            print("X_"+str(tree.key_feature), "<=", tree.threshold, "?", tree.ig)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
            
def precision(tree, actual):
        correct = 0
        for i in range(len(tree)):
            if tree[i] == actual[i]:
                correct += 1
        return correct/len(tree)

In [277]:
from sklearn.model_selection import train_test_split
train_set , test_set = train_test_split(data, test_size = 0.2, random_state = 5)

In [278]:
tree = DecisionTree(3)
tree.fit(train_set)
x = tree.predict(test_set.iloc[:,:-1].values)
tree.print_tree()

X_2 <= 1.9 ? 0.9340680553754911
 left:Setosa
 right:X_3 <= 1.7 ? 0.8001056702702978
  left:X_3 <= 1.4 ? 0.12136724088771567
    left:Versicolor
    right:Versicolor
  right:Virginica


In [279]:
tree.precision(x, test_set.iloc[:,-1].values)

0.9